In [5]:
!pip install -q rhino3dm shapely trimesh matplotlib pyvista


In [9]:
import rhino3dm
import numpy as np
from shapely.geometry import Polygon, Point, MultiPolygon, LineString
import matplotlib.pyplot as plt
import os
import pyvista as pv
import pandas as pd

# --- File Path Configuration ---
# Note: This path is remembered from your previous interactions.
rhino_path = r"C:\Users\papad\Documents\GitHub\Octopusie\Reference Files\rectangles3dcantilever - Copy.3dm"
if not os.path.exists(rhino_path):
    raise FileNotFoundError(f"File not found: {rhino_path}")

# --- Load Rhino Model and Extract Data ---
model = rhino3dm.File3dm.Read(rhino_path)
layers = {layer.Name.lower(): layer.Index for layer in model.Layers}

# Ensure required layers exist
if 'building' not in layers or ('column' not in layers and 'columns' not in layers):
    raise RuntimeError("Missing required layers: 'building' and 'column' or 'columns'.")

# Determine the correct column layer name
column_layer = 'columns' if 'columns' in layers else 'column'

building_volumes = []
imported_columns = []
max_z = 0.0 # Global maximum Z for the entire building footprint
wall_breps = [] # To store original building BREPs for visualization

for obj in model.Objects:
    layer_idx = obj.Attributes.LayerIndex
    geom = obj.Geometry

    if layer_idx == layers['building'] and geom.ObjectType == rhino3dm.ObjectType.Brep:
        bbox = geom.GetBoundingBox()
        base_pts = [
            [bbox.Min.X, bbox.Min.Y],
            [bbox.Max.X, bbox.Min.Y],
            [bbox.Max.X, bbox.Max.Y],
            [bbox.Min.X, bbox.Max.Y],
            [bbox.Min.X, bbox.Max.Y] # Ensure polygon is closed for Shapely
        ]
        poly = Polygon(base_pts)
        building_volumes.append(poly)
        wall_breps.append({'polygon': poly, 'bbox': bbox, 'brep': geom}) # Store the actual BREP
        max_z = max(max_z, bbox.Max.Z) # Update global max_z

    elif layer_idx == layers[column_layer] and geom.ObjectType == rhino3dm.ObjectType.Brep:
        bbox = geom.GetBoundingBox()
        center_x = (bbox.Min.X + bbox.Max.X) / 2
        center_y = (bbox.Min.Y + bbox.Max.Y) / 2
        imported_columns.append((center_x, center_y))

if not building_volumes:
    raise RuntimeError("No valid building geometry found on the 'building' layer.")

# --- User Input for Building Parameters ---
while True:
    try:
        num_floors = int(input("How many floors does the building have? (e.g., 2 for ground + 1 middle + roof): "))
        if num_floors < 1:
            raise ValueError
        break
    except ValueError:
        print("Please enter a valid positive integer for the number of floors.")

while True:
    try:
        wall_thickness = float(input("Enter desired wall thickness for the perimeter (e.g., 0.3): "))
        if wall_thickness <= 0:
            raise ValueError
        break
    except ValueError:
        print("Please enter a valid positive number for wall thickness.")

# --- Perimeter Calculation ---
combined_building_polygon = MultiPolygon(building_volumes)
try:
    exterior_perimeter = combined_building_polygon.buffer(wall_thickness, join_style=1) # join_style=1 for mitered joints
except Exception as e:
    print(f"Could not buffer the building outline. Error: {e}")
    exterior_perimeter = None

if exterior_perimeter and exterior_perimeter.geom_type == 'MultiPolygon':
    exterior_perimeter = max(exterior_perimeter.geoms, key=lambda p: p.area)

perimeter_line_coords = []
if exterior_perimeter:
    if exterior_perimeter.geom_type == 'Polygon':
        perimeter_line_coords = list(exterior_perimeter.exterior.coords)
    elif exterior_perimeter.geom_type == 'MultiPolygon':
        perimeter_line_coords = list(exterior_perimeter.geoms[0].exterior.coords)
    else:
        print("Warning: The buffered perimeter is not a Polygon or MultiPolygon. Cannot extract line coordinates.")

# --- Structural Grid Logic ---
detected_rooms = sorted([(poly, poly.area) for poly in building_volumes], key=lambda x: -x[1])
MaxS = 6.0
MinS = 3.0

columns = []
corrected_columns = []
available_imported = imported_columns.copy()
existing_columns = imported_columns.copy()
beams = [] # These are 2D beam definitions based on grid

for room_poly, _ in detected_rooms:
    minx, miny, maxx, maxy = room_poly.bounds
    width, height = maxx - minx, maxy - miny
    divisions_x = int(np.ceil(width / MaxS))
    divisions_y = int(np.ceil(height / MaxS))
    x_points = np.linspace(minx, maxx, divisions_x + 1)
    y_points = np.linspace(miny, maxy, divisions_y + 1)
    room_candidates = [(x, y) for x in x_points for y in y_points]

    for col in room_candidates:
        snap = False
        for imp_col in available_imported:
            if np.linalg.norm(np.array(col) - np.array(imp_col)) < MinS:
                corrected_columns.append(col)
                existing_columns.append(col)
                available_imported.remove(imp_col)
                snap = True
                break
        if not snap and all(np.linalg.norm(np.array(col) - np.array(exist_col)) >= MinS for exist_col in existing_columns):
            columns.append(col)
            existing_columns.append(col)

    for x in x_points:
        beams.append(((x, miny), (x, maxy)))
    for y in y_points:
        beams.append(((minx, y), (maxx, y)))

    for corner in list(room_poly.exterior.coords):
        if all(np.linalg.norm(np.array(corner) - np.array(exist_col)) >= MinS * 0.5 for exist_col in existing_columns):
            columns.append(corner)
            existing_columns.append(corner)

all_base_columns = columns + corrected_columns

# --- 2D Visualization (Commented out as per your request) ---
# fig, ax = plt.subplots(figsize=(12, 10))
# # ... (rest of 2D plotting code) ...
# plt.show()

# --- PyVista 3D Visualization (Main Plot) ---
plotter = pv.Plotter(title="3D Structural System")

# Define sizes for structural elements
column_width = 0.3
column_depth = 0.3
beam_width = 0.25
beam_depth = 0.35
slab_thickness = 0.20
slab_extension_buffer = max(column_width, column_depth) / 2 + 0.05

# Calculate story heights per room for accurate slab and column placement
room_story_heights = {}
for room_poly, _ in detected_rooms:
    room_specific_max_z = 0.0
    for wall_data in wall_breps:
        if wall_data['polygon'].equals(room_poly):
            room_specific_max_z = wall_data['bbox'].Max.Z
            break
    if room_specific_max_z == 0.0:
        room_specific_max_z = max_z # Fallback to global max_z if no specific wall data
    
    # Adjust for number of vertical sections (num_floors gives count of full stories + roof)
    num_vertical_sections = num_floors if num_floors > 0 else 1 
    story_height = room_specific_max_z / num_vertical_sections
    
    room_story_heights[room_poly.wkt] = story_height

# Define Z-levels for each floor in a structured way per room
floor_z_levels = {} 
for room_poly, _ in detected_rooms:
    room_specific_max_z = 0.0
    for wall_data in wall_breps:
        if wall_data['polygon'].equals(room_poly):
            room_specific_max_z = wall_data['bbox'].Max.Z
            break
    if room_specific_max_z == 0.0:
        room_specific_max_z = max_z

    current_room_floor_z = []
    
    # Slab 0: Ground Slab (always at Z=0)
    current_room_floor_z.append((0.0, slab_thickness))

    if num_floors > 0:
        story_height = room_story_heights.get(room_poly.wkt, max_z) # Use room-specific story height
        
        # Intermediate Slabs (up to num_floors - 1)
        for i in range(1, num_floors): 
            slab_base_z = i * story_height
            slab_top_z = slab_base_z + slab_thickness

            if slab_top_z > room_specific_max_z + 1e-6: # Prevent slab going above building max_z
                slab_top_z = room_specific_max_z
                slab_base_z = max(slab_base_z, slab_top_z - slab_thickness)

            if slab_top_z - slab_base_z < 1e-6: # Skip if slab is too thin
                continue
                
            current_room_floor_z.append((slab_base_z, slab_top_z))
        
        # Last Slab: Roof Slab
        roof_slab_top_z = room_specific_max_z 
        roof_slab_base_z = roof_slab_top_z - slab_thickness
        
        # Adjust roof slab if it overlaps with the last intermediate slab
        if current_room_floor_z:
            last_slab_top = current_room_floor_z[-1][1]
            if roof_slab_base_z < last_slab_top + 1e-6:
                roof_slab_base_z = last_slab_top
                roof_slab_top_z = roof_slab_base_z + slab_thickness
                if roof_slab_top_z > room_specific_max_z + 1e-6: # Ensure it doesn't exceed building height
                    roof_slab_top_z = room_specific_max_z
                    roof_slab_base_z = max(roof_slab_top_z - slab_thickness, last_slab_top)
        
        if roof_slab_top_z - roof_slab_base_z < 1e-6:
            pass
        else:
            current_room_floor_z.append((roof_slab_base_z, roof_slab_top_z))

    floor_z_levels[room_poly.wkt] = current_room_floor_z

# Add Slabs to the 3D plotter
for room_poly, _ in detected_rooms:
    room_floor_data = floor_z_levels.get(room_poly.wkt, [])
    if not room_floor_data:
        continue

    extended_room_poly = room_poly.buffer(slab_extension_buffer)
    
    for i, (slab_base_z, slab_top_z) in enumerate(room_floor_data):
        current_slab_thickness = slab_top_z - slab_base_z
        if current_slab_thickness < 1e-6:
            continue

        points_3d_for_slab_base = []
        poly_coords = list(extended_room_poly.exterior.coords)
        if poly_coords[0] != poly_coords[-1]:
            poly_coords.append(poly_coords[0])
            
        for x, y in poly_coords:
            points_3d_for_slab_base.append([x, y, 0.0])
        
        points_3d_for_slab_base = np.array(points_3d_for_slab_base)

        if len(points_3d_for_slab_base) < 3:
            continue

        slab_base_mesh = pv.PolyData(points_3d_for_slab_base)
        
        try:
            triangulated_faces = slab_base_mesh.delaunay_2d().faces
            if triangulated_faces is None or len(triangulated_faces) == 0:
                continue
            slab_base_mesh = pv.PolyData(points_3d_for_slab_base, faces=triangulated_faces)
        except Exception as e:
            print(f"Delaunay triangulation failed for a slab base ({room_poly.bounds}): {e}. Skipping slab.")
            continue

        slab_mesh = slab_base_mesh.extrude(vector=(0, 0, current_slab_thickness), capping=True)
        slab_mesh = slab_mesh.translate((0, 0, slab_base_z))
        
        plotter.add_mesh(slab_mesh, color='sienna', opacity=0.8, smooth_shading=True)

# --- Add Columns to the 3D plotter (as solid boxes, running through slabs) ---
column_data = [] # List to store column data for CSV

for x, y in all_base_columns:
    column_room_wkt = None
    for room_poly, _ in detected_rooms:
        if Point(x, y).within(room_poly):
            column_room_wkt = room_poly.wkt
            break
    if column_room_wkt is None:
        # If column is not strictly within a room, find the closest room
        min_dist = float('inf')
        for room_poly, _ in detected_rooms:
            dist = Point(x,y).distance(room_poly)
            if dist < min_dist:
                min_dist = dist
                column_room_wkt = room_poly.wkt
        if column_room_wkt is None:
              continue

    room_floor_data = floor_z_levels.get(column_room_wkt, [])
    if not room_floor_data or len(room_floor_data) <= 1:
        continue # Need at least two slabs to define a column segment

    num_slabs_in_room = len(room_floor_data)
    
    # Columns now span from the base of the current slab to the base of the next slab
    for i in range(num_slabs_in_room - 1):
        col_base_z = room_floor_data[i][0]   # Base of current slab
        col_top_z = room_floor_data[i+1][0]  # Base of next slab

        if col_top_z <= col_base_z + 1e-6: # Skip if column segment is too small or inverted
            continue

        col_min_x = x - column_width / 2
        col_max_x = x + column_width / 2
        col_min_y = y - column_depth / 2
        col_max_y = y + column_depth / 2
        
        column_box = pv.Box([col_min_x, col_max_x, col_min_y, col_max_y, col_base_z, col_top_z])
        plotter.add_mesh(column_box, color='blue', smooth_shading=True, label='Columns') # Columns are solid boxes
        
        column_data.append({
            'X': x, 
            'Y': y, 
            'Z_Base': col_base_z, 
            'Z_Top': col_top_z, 
            'Width': column_width, 
            'Depth': column_depth,
            'Floor_Level': i + 1
        })

# --- Add Beams to the 3D plotter (as solid boxes, restoring their width) ---
beam_data = [] # List to store beam data for CSV

for (x1_2d, y1_2d), (x2_2d, y2_2d) in beams: # These are 2D beam definitions
    full_beam_line_2d = LineString([(x1_2d, y1_2d), (x2_2d, y2_2d)])

    # Find all rooms that this 2D beam line intersects
    intersecting_rooms_info = []
    for room_poly, _ in detected_rooms:
        if room_poly.intersects(full_beam_line_2d.buffer(1e-3)): # Use a small buffer for robust intersection
            intersecting_rooms_info.append(room_poly)

    # For each intersecting room, generate beams at all its floor levels
    for room_poly in intersecting_rooms_info:
        room_wkt = room_poly.wkt
        room_specific_floor_data = floor_z_levels.get(room_wkt, [])

        if not room_specific_floor_data:
            continue

        # Get the actual intersection segment of the beam within this specific room
        segment_in_room = full_beam_line_2d.intersection(room_poly)

        segments_to_draw = []
        if segment_in_room.geom_type == 'LineString':
            segments_to_draw = [segment_in_room]
        elif segment_in_room.geom_type == 'MultiLineString':
            segments_to_draw = list(segment_in_room.geoms)
        else:
            continue # No valid line intersection

        for segment in segments_to_draw:
            if not segment.is_empty and segment.length > 1e-6:
                seg_x1, seg_y1 = segment.coords[0]
                seg_x2, seg_y2 = segment.coords[-1]

                # Now, for each actual geometric segment of the beam *within this room*,
                # generate a 3D beam for each floor level in that room
                for i, (slab_base_z, slab_top_z) in enumerate(room_specific_floor_data):
                    # Beams are typically below the slab, so their top Z is the slab's base Z
                    
                    beam_top_z = slab_base_z 
                    beam_base_z = beam_top_z - beam_depth

                    # Ensure beam doesn't go below ground or is too thin
                    if beam_base_z < 0 and i == 0: # For ground level, cap at Z=0
                        beam_base_z = 0
                        if beam_top_z <= 0: # If slab itself is at/below 0, no beam
                            continue
                    elif beam_top_z <= beam_base_z + 1e-6:
                        continue # Skip if beam is too thin or inverted

                    # Determine beam bounding box based on orientation
                    if abs(seg_x1 - seg_x2) > abs(seg_y1 - seg_y2): # More horizontal
                        beam_min_x = min(seg_x1, seg_x2)
                        beam_max_x = max(seg_x1, seg_x2)
                        beam_center_y = (seg_y1 + seg_y2) / 2
                        beam_min_y = beam_center_y - beam_width / 2
                        beam_max_y = beam_center_y + beam_width / 2
                    else: # More vertical
                        beam_min_y = min(seg_y1, seg_y2)
                        beam_max_y = max(seg_y1, seg_y2)
                        beam_center_x = (seg_x1 + seg_x2) / 2
                        beam_min_x = beam_center_x - beam_width / 2
                        beam_max_x = beam_center_x + beam_width / 2

                    beam_box = pv.Box([beam_min_x, beam_max_x, beam_min_y, beam_max_y, beam_base_z, beam_top_z]) # Beams are solid boxes

                    # Color beams differently based on whether they are roof beams
                    is_roof_slab_level = (i == len(room_specific_floor_data) - 1) and num_floors > 0
                    color = 'green' if is_roof_slab_level else 'orange' # Use same colors as before for distinction
                    plotter.add_mesh(beam_box, color=color, smooth_shading=True, label='Beams') # Added smooth_shading back
                    
                    beam_data.append({
                        'Start_X': seg_x1, 
                        'Start_Y': seg_y1, 
                        'End_X': seg_x2, 
                        'End_Y': seg_y2, 
                        'Z_Base': beam_base_z, # Actual base of the 3D beam
                        'Z_Top': beam_top_z,   # Actual top of the 3D beam
                        'Width': beam_width, 
                        'Depth': beam_depth,
                        'Floor_Level': i + 1
                    })

# --- Visualize Original Building Walls (as semi-transparent BREPs) ---
def mesh_brep(brep, mesh_type=rhino3dm.MeshType.Any):
    meshes = []
    for face in brep.Faces:
        try:
            m = face.GetMesh(mesh_type)
            if m: meshes.append(m)
        except:
            continue
    return meshes

for wall_data in wall_breps:
    brep_geom = wall_data['brep']
    meshes = mesh_brep(brep_geom)
    for mesh in meshes:
        pts = [(v.X, v.Y, v.Z) for v in mesh.Vertices]
        faces = []
        for f in mesh.Faces:
            if len(f) == 4: # Quad face
                idxs = (f[0], f[1], f[2], f[3])
            else: # Tri face
                idxs = (f[0], f[1], f[2])
            faces.append((len(idxs),) + idxs) # PyVista expects (num_points_in_face, p1_idx, p2_idx, ...)
        
        # Flatten the list of faces for PyVista
        faces_flat = [i for face in faces for i in face]
        
        pv_mesh = pv.PolyData(pts, faces_flat)
        plotter.add_mesh(pv_mesh, color='lightgray', opacity=0.3)

plotter.show_grid()
plotter.show()

# --- Create and Save CSVs ---
output_dir = "structural_data"
os.makedirs(output_dir, exist_ok=True)

columns_df = pd.DataFrame(column_data)
beams_df = pd.DataFrame(beam_data)

columns_csv_path = os.path.join(output_dir, "columns_data.csv")
beams_csv_path = os.path.join(output_dir, "beams_data.csv")

columns_df.to_csv(columns_csv_path, index=False)
beams_df.to_csv(beams_csv_path, index=False)

print(f"\nColumn data saved to: {columns_csv_path}")
print(f"Beam data saved to: {beams_csv_path}")

# --- Visualize CSVs in 3D (Separate Plotter for CSV data) ---
print("\nVisualizing saved CSVs from CSV Data:")

plotter_csv = pv.Plotter(title="3D Visualization from CSV Data")

# Visualize Columns from CSV (as lines)
if not columns_df.empty:
    for index, column in columns_df.iterrows():
        # Create a line representation for the column from CSV data
        start_point_3d = np.array([column['X'], column['Y'], column['Z_Base']])
        end_point_3d = np.array([column['X'], column['Y'], column['Z_Top']])

        points = np.array([start_point_3d, end_point_3d])
        lines = np.array([[2, 0, 1]]) # Define a single line segment connecting point 0 and 1

        column_line_mesh = pv.PolyData(points, lines=lines)
        # Using render_lines_as_tubes for better visibility of lines in 3D
        plotter_csv.add_mesh(column_line_mesh, color='blue', line_width=5, render_lines_as_tubes=True, label='Columns (from CSV)') # Columns are lines

# Visualize Beams from CSV (as lines)
if not beams_df.empty:
    for index, beam in beams_df.iterrows():
        # Represent beam as a line at its top Z (bottom of slab)
        start_point_3d = np.array([beam['Start_X'], beam['Start_Y'], beam['Z_Top']])
        end_point_3d = np.array([beam['End_X'], beam['End_Y'], beam['Z_Top']])

        # Create a PolyData object for the line segment
        points = np.array([start_point_3d, end_point_3d])
        lines = np.array([[2, 0, 1]]) # Define a single line segment connecting point 0 and 1

        line_mesh = pv.PolyData(points, lines=lines)
        # Using render_lines_as_tubes for better visibility of lines in 3D
        plotter_csv.add_mesh(line_mesh, color='red', line_width=5, render_lines_as_tubes=True, label='Beams (from CSV)')

plotter_csv.show_grid()
plotter_csv.show()

Widget(value='<iframe src="http://localhost:55826/index.html?ui=P_0x1f857bcf4d0_12&reconnect=auto" class="pyvi…


Column data saved to: structural_data\columns_data.csv
Beam data saved to: structural_data\beams_data.csv

Visualizing saved CSVs from CSV Data:


Widget(value='<iframe src="http://localhost:55826/index.html?ui=P_0x1f8aaf89590_13&reconnect=auto" class="pyvi…